# ==============================================================
# DATA ANALYSIS

### This is a user-interactive notebook dedicated to the visual analysis of the data and the extraction of necessary information for data preprocessing.
**OUTPUT:** "{db_name}_prep.json" file in the format for running the "data_preprocessing.py" script
# ==============================================================

In [ ]:
# Imports
import pandas as pd
import json
pd.set_option('display.max_rows', 10)
JSON_OBJ = {}

## ------------------------  0. DATA FILE INFO  ------------------------

> **[INPUT]** DATA FILE PATH AND DATABASE NAME

In [ ]:
DATA_PATH = ''
DB_NAME = ''

## ------------------------  1. EMPTY COLUMNS  ------------------------

**[VISUAL ANALYSIS]** _IDENTIFY_ EMPTY COLUMNS TO DROP
<br>*adjust separator if necessary*

In [ ]:
db_orig = pd.read_csv(DATA_PATH, header=0, sep=',')
db_orig.head()

>**[INPUT]:** EMPTY COLUMNS
<br>List of the **names** from columns to drop
<br>default: **_emptCols = []** 

In [ ]:
_emptCols = []
#########################################
JSON_OBJ['_emptCols'] = _emptCols
print('_emptCols = {}'.format(_emptCols))

## ------------------------  2. UNTREACEBLE MISSING VALUES  -----------------

**[VISUAL ANALYSIS]** _IDENTIFY_ UNTRACEABLE MISSING VALUES


In [ ]:
print('================ Attribute-Values analysis')
it=0
for (columnName, columnData) in db_orig.iteritems():
    print('\n----')
    print('Column{}:{}'.format(it,columnName))
    print('data-type: {}'.format(columnData.dtype))
    print('#unique: {}'.format(len(columnData.unique())))
    print('Unique values: {}'.format(columnData.unique()))
    it += 1

>**[INPUT]** UNTREACEBLE MISSING VALUES ID
<br>Dictionary {'attribute-name': ['missing-value']} for pandas-untreaceble missing values:
<br>default: **_dictMV = {}**
<br>! *attention*: dict{ str : list}

In [ ]:
_dictMV = {}
######################################
JSON_OBJ['_dictMV'] = _dictMV
print('_dictMV = {}'.format(_dictMV))

## ------------------------  3. FEATURES SELECTION  ------------------------

**[VISUAL ANALYSIS]** _IDENTIFY_ THE COLUMNS NOT TO USE
<br>*look at the db info and at info on missing values if it helps*

In [ ]:
if _dictMV:
    mv_values = []
    for attr, values in _dictMV.items():
        for value in values:
            if not value in mv_values:
                mv_values.append(value)
# reaload db with adjusted missing values and drop empty cols
db_new = pd.read_csv(DATA_PATH, header=0, sep=';', na_values=mv_values)
if _emptCols:
    db_new.drop(columns=_emptCols,inplace=True)

print('===== DB INFO:') 
display(db_new.info())

print('\n===== MISSING VALUES:')
mv = db_new.isnull().sum().sum()
if mv == 0:
    print('! No missing values')
else:
    mv_cols = db_new.columns[db_new.isnull().any()].tolist()
    print('# missing-values: {}'.format(mv))
    print('\n#mv by attribute:')
    for col in mv_cols:
        print('{} = {}'.format(col,db_new[col].isnull().sum()))

>**[INPUT]** COLUMNS' *NOT* TO USE
<br>List of the **names** of the columns to be dropped
<br>default: **_colsNot2use = [ ]**

In [ ]:
_colsNot2use = [ ]
###############################################
JSON_OBJ['_colsNot2use'] = _colsNot2use
print('_colsNot2use = {}'.format(_colsNot2use))

## ------------------------  4. SURVIVAL FEATURES ------------------------

**[VISUAL ANALYSIS]** _IDENTIFY_ THE SURVIVAL VARIABLES NAMES

In [ ]:
print('Features:')
for col in db_new.columns:
    print(col)

>**[INPUT]** SURVIVAL ATTRIBUTES NAMES
<br> Write the attributes names: survival time (survTime_name) and event/cens/status (survEvent_name)
<br> **_mandatory_**

In [ ]:
survivalTime_name = ''
survivalEvent_name = ''
########################################################
_survivalAttr = {'survivalTime_name':survivalTime_name,
                'survivalEvent_name':survivalEvent_name}
JSON_OBJ['_survivalAttr'] = _survivalAttr
print('_survivalAttr = {}'.format(_survivalAttr))

## ------------------------  5. EVENT FEATURE ADJUSTMENT  ------------------------

**[VISUAL ANALYSIS]** _IDENTIFY_ IF EVENT COLUMN NEEDS ADJUSTMENT
<br>*the feature needs adjustment if it is not FALSE/TRUE nor 0/1*

In [ ]:
print('===== EVENT VARIABLE < {} >:'.format(survivalEvent_name))
print('Type: {}'.format(db_new[survivalEvent_name].dtype))
print('Unique values: {}'.format(db_new[survivalEvent_name].unique()))

>**[INPUT]** SURVIVAL STATUS REPRESENTATION
<br>Identify the censoring values (censValue) and the event-occurrence values (eventValue)
<br>default: **censValue/eventValue = [ ]**

In [ ]:
censValue = []
eventValue = []
##############################################
_statusRepr = {'censValue':censValue,
              'eventValue':eventValue}
JSON_OBJ['_statusRepr'] = _statusRepr
print('_statusRepr = {}'.format(_statusRepr))

## ------------------------  6. CATEGORY IMPUTATION  ------------------------

**[VISUAL ANALYSIS]** _IDENTIFY_ IF THERE ARE CATEGORIES TO INPUT ON MISSING-VALUES
<br>*use information on missing values if necessary*

In [ ]:
print('===== OVERALL MINSSING CASES INFO: ')
mv = db_new.isnull().sum().sum()
if mv == 0:
    print('\n! No missing cases')
else:
    print('# missing-values = {}'.format(mv))
    print('# rows_mv/rows_total: {}/{}'.format(db_new.isnull().any(axis='columns').sum(), db_new.shape[0]))
    print('\n# mv by attribute:')
    for col in db_new.columns[db_new.isnull().any()].tolist():
        print('- Attribute {}: #{}'.format(col,db_new[col].isnull().sum()))

>**[INPUT]** NaN VALUES FOR INPUTTING
<br>Disctionary {columnName : value-to-input} to input on missing values
<br>default: **nan2replace = {}**

In [ ]:
_nan2replace = {}
###############################################
JSON_OBJ['_nan2replace'] = _nan2replace
print('_nan2replace = {}'.format(_nan2replace))

## ------------------------  7. DTYPES DEFINITION  ------------------------

**[VISUAL ANALYSIS]** _IDENTIFY_ THE COLUMNS' DTYPES TO CHANGE
<br>*use db.info if necessary*

In [ ]:
display(db_new.info())

>**[INPUT]** NaN VALUES FOR INPUTTING
<br>Write the columns' **names** for each type: category (colsCtg) or boolean (colsBool):
<br>default: **colsCtg/colsBool: []**
<br>! *attention*: features not encompassed will be considered numerical for discretization

In [ ]:
colsCtg = []
colsBool = []   # obs.: [true/false] or [0/1] values
###############################################################
booleans = dict.fromkeys(colsBool,'bool')
categorical = dict.fromkeys(colsCtg,'category')
_colsType = {**booleans, **categorical}
_cols2disc = list(set(colsCtg+colsBool) ^ set(db_new.columns))
_cols2disc.remove(_survivalAttr['survivalTime_name'])          # remove survival-time feature from discretization names
if _survivalAttr['survivalEvent_name'] in _cols2disc:          # remove survival-event feature from discretization names
    _cols2disc.remove(_survivalAttr['survivalEvent_name'])
JSON_OBJ['_colsType'] = _colsType
JSON_OBJ['_cols2disc'] = _cols2disc
print('_colsType = {}'.format(_colsType))
print('_cols2disc = {}'.format(_cols2disc))

# !! SAVING !!

In [ ]:
with open('{}_prep.json'.format(DB_NAME)) as f:
    json.dump(JSON_OBJ,f)